# Docker Compose

https://docs.docker.com/compose/compose-file/#volume-configuration-reference

**Compose is a tool for defining and running multi-container Docker applications.** With Compose, you use a YAML file to configure your application’s services. Then, with a single command, you create and start all the services from your configuration. To learn more about all the features of Compose, see the list of features.

Compose works in all environments: production, staging, development, testing, as well as CI workflows. You can learn more about each case in Common Use Cases.

Using Compose is basically a three-step process:
1. Define your app’s environment with a Dockerfile so it can be reproduced anywhere.
2. Define the services that make up your app in docker-compose.yml so they can be run together in an isolated environment.
3. Run docker-compose up and Compose starts and runs your entire app.

Compose has commands for managing the whole lifecycle of your application:
- Start, stop, and rebuild services
- View the status of running services
- Stream the log output of running services
- Run a one-off command on a service

## Features

**Multiple isolated environments on a single host**

Compose uses a project name to isolate environments from each other. You can make use of this project name in several different contexts:
- on a dev host, to create multiple copies of a single environment, such as when you want to run a stable copy for each feature branch of a project
- on a CI server, to keep builds from interfering with each other, you can set the project name to a unique build number
- on a shared host or dev host, to prevent different projects, which may use the same service names, from interfering with each other

The default project name is the basename of the project directory. You can set a custom project name by using the -p command line option or the COMPOSE_PROJECT_NAME environment variable.

**Preserve volume data when containers are created**

Compose preserves all volumes used by your services. When docker-compose up runs, if it finds any containers from previous runs, it copies the volumes from the old container to the new container. This process ensures that any data you’ve created in volumes isn’t lost.

If you use docker-compose on a Windows machine, see Environment variables and adjust the necessary environment variables for your specific needs.

**Only recreate containers that have changed**

Compose caches the configuration used to create a container. When you restart a service that has not changed, Compose re-uses the existing containers. Re-using containers means that you can make changes to your environment very quickly.

**Variables and moving a composition between environments**

Compose supports variables in the Compose file. You can use these variables to customize your composition for different environments, or different users. See Variable substitution for more details.

You can extend a Compose file using the extends field or by creating multiple Compose files. See extends for more details.

## Why care about Docker-compose

Before we get into the technical details let's discuss why a programmer should even care about docker-compose in the first place. Here are some reasons why developers should consider implementing Docker in their work.

**Portability:**

Docker Compose lets you bring up a complete development environment with only one command: docker-compose up, and tear it down just as easily using docker-compose down. This allows us developers to keep our development environment in one central place and helps us to easily deploy our applications.

**Testing:**

Another great feature of Compose is its support for running unit and E2E tests in a quick a repeatable fashion by putting them in their own environments. That means that instead of testing the application on your local/host OS, you can run an environment that closely resembles the production circumstances.

**Multiple isolated environments on a single host:**
    
Compose uses project names to isolate environments from each other which brings the following benefits:

You can run multiple copies of the same environment on one machine
It prevents different projects and service from interfering with each other

## Common use cases

Now that you know why Compose is useful and where it can improve the workflow of us developers let's take a look at some common use cases.

**Single host deployments:**

Compose was traditionally focused on development and testing but can now be used to deploy and manage a whole deployment of containers on a single host system.

**Development environments:**

Compose provides the ability to run your applications in an isolated environment that can run on any machine with Docker installed. This makes it very easy to test your application and provides a way to work as close to the production environment as possible.

The Compose file manages all the dependencies (databases, queues, caches, etc) of the application and can create every container using a single command.

**Automated testing environments:**

An important part of Continues integration and the whole development process is the automated testing suite which requires an environment in which the tests can be executed. Compose provides a convenient way to create and destroy isolated testing environments that are close to your production environment.

## Example: why we need compose

Naredimo mapo `visits`

- Naredimo package.json in index.js

```json
{
  "dependencies": {
    "express": "*",
    "redis": "2.8.0"
  },
  "scripts": {
    "start": "node index.js"
  }
}


```

```javascript
const express = require("express");
const redis = require("redis");
const process = require("process");

const app = express();
const client = redis.createClient({
  host: "redis-server",
  port: 6379
});
client.set("visits", 0);

app.get("/", (req, res) => {
  //process.exit(0)
  client.get("visits", (err, visits) => {
    res.send("Number of visits " + visits);
    client.set("visits", parseInt(visits) + 1);
  });
});

app.listen(8081, () => {
  console.log("listening on port 8081");
});
```

Naredimo Dockerfile:

```Dockerfile
FROM node:alpine

WORKDIR '/app'

COPY package.json .
RUN npm install
COPY . .

CMD ["npm","start"]
```

Zgradimo sliko:
    
    docker build -t leon11sj/visits . 

Pokažemo na roko kako lahko zaženemo 2 kontejnerja.

Morali bi ročno narediti omrežje, kar ni najboljša praksa.

    docker network create --driver bridge --attachable redis-app

    docker run -d --network redis-app --name redis-server redis
    docker run -d --network redis-app -p 80:8081 --name node-app --restart on-failure leon11sj/visits

Odstranimo:
    
    docker stop node-app redis-server
    docker rm node-app redis-server
    docker network rm redis-app

Boljša praksa pri več storitvah je uporabiti docker-compose.

```yml
version: "3"
services:
  redis-server:
    image: "redis"
  node-app:
    restart: on-failure
    build: .
    ports:
      - "80:8081"
```

    docker-compose up
    
Rebuild the image

    docker-compose up --build 

Pokažemo da compose nardi novi network, da združi naše kontejnerje skupaj

Zaženemo kontejnerje v ozadju:
    
    docker-compose up -d 

Ustavimo:
    
    docker-compose down

Pokažemo kako kontejner pade, in kako ga lahko resetiramo
- odkomentiramo //process.exit(0)
- dodamao v docker-compose  restart: on-failure
- pokažemo različne stop kode (npr, 1,2)

<table>
  <thead>
    <tr>
      <th style="text-align: left">Flag</th>
      <th style="text-align: left">Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="text-align: left"><code class="highlighter-rouge">no</code></td>
      <td style="text-align: left">Do not automatically restart the container. (the default)</td>
    </tr>
    <tr>
      <td style="text-align: left"><code class="highlighter-rouge">on-failure</code></td>
      <td style="text-align: left">Restart the container if it exits due to an error, which manifests as a non-zero exit code.</td>
    </tr>
    <tr>
      <td style="text-align: left"><code class="highlighter-rouge">always</code></td>
      <td style="text-align: left">Always restart the container if it stops. If it is manually stopped, it is restarted only when Docker daemon restarts or the container itself is manually restarted. (See the second bullet listed in <a href="#restart-policy-details">restart policy details</a>)</td>
    </tr>
    <tr>
      <td style="text-align: left"><code class="highlighter-rouge">unless-stopped</code></td>
      <td style="text-align: left">Similar to <code class="highlighter-rouge">always</code>, except that when the container is stopped (manually or otherwise), it is not restarted even after Docker daemon restarts.</td>
    </tr>
  </tbody>
</table>

Pokaže samo kontejnerje, ki so zraven datoteke katero zaganjamo

    docker-compose ps

## Get started with Docker Compose

On this page you build a simple Python web application running on Docker Compose. The application uses the Flask framework and maintains a hit counter in Redis. While the sample uses Python, the concepts demonstrated here should be understandable even if you’re not familiar with it.

Define the application dependencies.

Create a directory for the project:

    mkdir composetest
    cd composetest

Create a file called `app.py` in your project directory and paste this in:

In [ ]:
import time

import redis
from flask import Flask

app = Flask(__name__)
cache = redis.Redis(host='redis', port=6379)


def get_hit_count():
    retries = 5
    while True:
        try:
            return cache.incr('hits')
        except redis.exceptions.ConnectionError as exc:
            if retries == 0:
                raise exc
            retries -= 1
            time.sleep(0.5)


@app.route('/')
def hello():
    count = get_hit_count()
    return 'Hello World! I have been seen {} times.\n'.format(count)

In this example, redis is the hostname of the redis container on the application’s network. We use the default port for Redis, 6379.

> **Handling transient errors:** Note the way the get_hit_count function is written. This basic retry loop lets us attempt our request multiple times if the redis service is not available. This is useful at startup while the application comes online, but also makes our application more resilient if the Redis service needs to be restarted anytime during the app’s lifetime. In a cluster, this also helps handling momentary connection drops between nodes.

Create another file called `requirements.txt` in your project directory and paste this in:

    flask
    redis

### Create a Dockerfile

In this step, you write a Dockerfile that builds a Docker image. The image contains all the dependencies the Python application requires, including Python itself.

In your project directory, create a file named `Dockerfile` and paste the following:

```Dockerfile
FROM python:3.7-alpine
WORKDIR /code
ENV FLASK_APP app.py
ENV FLASK_RUN_HOST 0.0.0.0
RUN apk add --no-cache gcc musl-dev linux-headers
COPY requirements.txt requirements.txt
RUN pip install -r requirements.txt
EXPOSE 5000
COPY . .
CMD ["flask", "run"]
```

This tells Docker to:
- Build an image starting with the Python 3.7 image.
- Set the working directory to /code.
- Set environment variables used by the flask command.
- Install gcc and other dependencies
- Copy requirements.txt and install the Python dependencies.
- Add metadata to the image to describe that the container is listening on port 5000
- Copy the current directory . in the project to the workdir . in the image.
- Set the default command for the container to flask run.
- For more information on how to write Dockerfiles, see the Docker user guide and the Dockerfile reference.

### Define services in a Compose file

Create a file called `docker-compose.yml` in your project directory and paste the following:

```yml
version: '3'
services:
  web:
    build: ./composetest
    ports:
      - "80:5000"
  redis:
    image: "redis:alpine"
```

This Compose file defines two services: web and redis.

**Web service**

The web service uses an image that’s built from the Dockerfile in the current directory. It then binds the container and the host machine to the exposed port, 5000. This example service uses the default port for the Flask web server, 5000.

**Redis service**

The redis service uses a public Redis image pulled from the Docker Hub registry.

### Build and run your app with Compose

From your project directory, start up your application by running docker-compose up

     docker-compose up

Compose pulls a Redis image, builds an image for your code, and starts the services you defined. In this case, the code is statically copied into the image at build time.

<hr>

Enter `http://<IP>:5000/` in a browser to see the application running.

If you’re using Docker natively on Linux, Docker Desktop for Mac, or Docker Desktop for Windows, then the web app should now be listening on port 5000 on your Docker daemon host. Point your web browser to http://localhost:5000 to find the Hello World message. If this doesn’t resolve, you can also try http://127.0.0.1:5000.

If you’re using Docker Machine on a Mac or Windows, use docker-machine ip MACHINE_VM to get the IP address of your Docker host. Then, open http://MACHINE_VM_IP:5000 in a browser.

You should see a message in your browser saying:

<hr>

Switch to another terminal window, and type docker image ls to list local images.

Listing images at this point should return redis and web.

    docker image ls

You can inspect images with `docker inspect <tag or id>`

Stop the application, either by running `docker-compose down` from within your project directory in the second terminal, or by hitting CTRL+C in the original terminal where you started the app.

### Edit the Compose file to add a bind mount

Edit `docker-compose.dev.yml` in your project directory to add a bind mount for the web service:

```yml
version: '3'
services:
  web:
    build: ./composetest
    ports:
      - "80:5000"
    volumes:
      - ./composetest:/code
    environment:
      FLASK_ENV: development
  redis:
    image: "redis:alpine"
```

The new volumes key mounts the project directory (current directory) on the host to /code inside the container, allowing you to modify the code on the fly, without having to rebuild the image. The environment key sets the FLASK_ENV environment variable, which tells flask run to run in development mode and reload the code on change. This mode should only be used in development.

### Re-build and run the app with Compose

From your project directory, type docker-compose up to build the app with the updated Compose file, and run it.

    docker-compose -f docker-compose.dev.yml up

Check the Hello World message in a web browser again, and refresh to see the count increment.

### Update the application

Because the application code is now mounted into the container using a volume, you can make changes to its code and see the changes instantly, without having to rebuild the image.

Change the greeting in app.py and save it. For example, change the Hello World! message to Hello from Docker!:

    return 'Hello from Docker! I have been seen {} times.\n'.format(count)

Refresh the app in your browser. The greeting should be updated, and the counter should still be incrementing.

### Experiment with some other commands

If you want to run your services in the background, you can pass the -d flag (for “detached” mode) to docker-compose up and use docker-compose ps to see what is currently running:

    docker-compose up -d

    docker-compose ps

The docker-compose run command allows you to run one-off commands for your services. For example, to see what environment variables are available to the web service:

    docker-compose run web env

If you started Compose with docker-compose up -d, stop your services once you’ve finished with them:

    docker-compose stop

You can bring everything down, removing the containers entirely, with the down command. Pass --volumes to also remove the data volume used by the Redis container:

    docker-compose down --volumes

At this point, you have seen the basics of how Compose works.

## Networking in Compose

By default Compose sets up a single network for your app. Each container for a service joins the default network and is both reachable by other containers on that network, and discoverable by them at a hostname identical to the container name.

> Note: Your app’s network is given a name based on the “project name”, which is based on the name of the directory it lives in. You can override the project name with either the --project-name flag or the COMPOSE_PROJECT_NAME environment variable.

    version: "3"
    services:
      web:
        build: .
        ports:
          - "8000:8000"
      db:
        image: postgres
        ports:
          - "8001:5432"

Each container can now look up the hostname web or db and get back the appropriate container’s IP address. For example, web’s application code could connect to the URL postgres://db:5432 and start using the Postgres database.

It is important to note the distinction between HOST_PORT and CONTAINER_PORT. In the above example, for db, the HOST_PORT is 8001 and the container port is 5432 (postgres default). Networked service-to-service communication uses the CONTAINER_PORT. When HOST_PORT is defined, the service is accessible outside the swarm as well.

Within the web container, your connection string to db would look like postgres://db:5432, and from the host machine, the connection string would look like postgres://{DOCKER_IP}:8001.

### Specify custom networks

Instead of just using the default app network, you can specify your own networks with the top-level networks key. This lets you create more complex topologies and specify custom network drivers and options. You can also use it to connect services to externally-created networks which aren’t managed by Compose.

Each service can specify what networks to connect to with the service-level networks key, which is a list of names referencing entries under the top-level networks key.

By default Compose sets up a single network for each container. Each container is automatically joining the default network which makes them reachable by both other containers on the network, and discoverable by the hostname defined in the Compose file.


https://docs.docker.com/compose/compose-file/#network-configuration-reference

    version: '3'
    services:
      web:
        build: ./composetest
        ports:
          - "80:5000"
        volumes:
          - ./composetest:/code
        environment:
          FLASK_ENV: development
        networks:
          - backend
          - frontend
      redis:
        image: "redis:alpine"
        networks:  
          - backend


    networks:
      frontend:
      backend:

    docker-compose -f docker-compose.v3.yml up -d --build

    docker-compose logs -f

    docker-compose down --volumes

# Production

## Gunicorn

### Creating the WSGI Entry Point

Moving along, for production environments, let's add Gunicorn, a production-grade WSGI server, to the requirements file:

    gunicorn==20.0.4

Next, let’s create a file that will serve as the entry point for our application. This will tell our Gunicorn server how to interact with the application.

Let’s call the file `wsgi.py`:

In [ ]:
from app import app

if __name__ == "__main__":
    app.run()

### Production Dockerfile

To use this file, create a new Dockerfile called Dockerfile.prod for use with production builds:

```Dockerfile
###########
# BUILDER #
###########

# pull official base image
FROM python:3.8.1-slim-buster as builder

# set work directory
WORKDIR /usr/src/app

# set environment variables
ENV PYTHONDONTWRITEBYTECODE 1
ENV PYTHONUNBUFFERED 1

# install system dependencies
RUN apt-get update && \
    apt-get install -y --no-install-recommends gcc

# install python dependencies
COPY ./requirements.txt .
RUN pip wheel --no-cache-dir --no-deps --wheel-dir /usr/src/app/wheels -r requirements.txt


#########
# FINAL #
#########

# pull official base image
FROM python:3.8.1-slim-buster

# create directory for the app user
RUN mkdir -p /home/app

# create the appropriate directories
ENV HOME=/home/app
ENV APP_HOME=/home/app/web
RUN mkdir $APP_HOME
WORKDIR $APP_HOME

# install dependencies
RUN apt-get update && apt-get install -y --no-install-recommends netcat
COPY --from=builder /usr/src/app/wheels /wheels
COPY --from=builder /usr/src/app/requirements.txt .
RUN pip install --upgrade pip
RUN pip install --no-cache /wheels/*

# copy project
COPY . $APP_HOME

# run entrypoint.prod.sh
ENTRYPOINT ["python", "/home/app/web/wsgi.py"]
```

Here, we used a Docker multi-stage build to reduce the final image size. Essentially, builder is a temporary image that's used for building the Python wheels. The wheels are then copied over to the final production image and the builder image is discarded.

    docker build -t leon11sj/counter -f ./Dockerfile.prod .

### Preizkusimo delovanje

Since we still want to use Flask's built-in server in development, create a new compose file called `docker-compose.prod.yml` for production:

```yml
version: '3.7'
services:
  web:
    image: leon11sj/counter
    entrypoint: ["gunicorn", "--bind", "0.0.0.0:5000", "wsgi:app"]
    ports:
      - "80:5000"
    networks:
      - backend
      - frontend
    depends_on:
      - redis
  redis:
    image: "redis:alpine"
    networks:  
      - backend
    

networks:
  frontend:
  backend:
```

    docker-compose -f docker-compose.prod.yml up

## Nginx as a reverse proxy

Next, let's add Nginx into the mix to act as a reverse proxy for Gunicorn to handle client requests as well as serve up static files.

Add the service to docker-compose.prod.yml:

      nginx:
        build: 
          context: ./nginx
          dockerfile: Dockerfile
        ports:
          - 80:8080
        depends_on:
          - web
        networks:
          - frontend

Then,  create the following files and folders:

    └── nginx
        ├── Dockerfile
        └── nginx.conf

`Dockerfile:`

    FROM nginx:1.17-alpine

    RUN rm /etc/nginx/conf.d/default.conf
    COPY nginx.conf /etc/nginx/conf.d

`nginx.conf:`

    upstream flask {
        server web:5000;
    }

    server {

        listen 8080;

        location / {
            proxy_pass http://flask;
            proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
            proxy_set_header Host $host;
            proxy_redirect off;
        }

    }

https://www.patricksoftwareblog.com/how-to-configure-nginx-for-a-flask-web-application/

Then, update the web service, in docker-compose.prod.yml, replacing ports with expose:

```yml
version: '3.7'
services:
  nginx:
    build: 
      context: ./nginx
      dockerfile: Dockerfile
    ports:
      - 80:8080
    depends_on:
      - web
    networks:
      - frontend
  web:
    image: leon11sj/counter
    entrypoint: ["gunicorn", "--bind", "0.0.0.0:5000", "wsgi:app"]
    expose:
      - 5000
    networks:
      - backend
      - frontend
    depends_on:
      - redis
  redis:
    image: "redis:alpine"
    networks:  
      - backend
    

networks:
  frontend:
  backend:

```

    docker-compose -f docker-compose.prod.yml up --build

## Persistent Redis data

https://docs.docker.com/compose/compose-file/#volume-configuration-reference

Prilagodimo compose file:

```yml
version: '3.7'
services:
  nginx:
    build: 
      context: ./nginx
      dockerfile: Dockerfile
    ports:
      - 80:8080
    depends_on:
      - web
    networks:
      - frontend
  web:
    image: leon11sj/counter
    entrypoint: ["gunicorn", "--bind", "0.0.0.0:5000", "wsgi:app"]
    expose:
      - 5000
    networks:
      - backend
      - frontend
    depends_on:
      - redis
  redis:
    image: "redis:alpine"
    volumes:
      - redis-data:/data
    networks:  
      - backend
    

networks:
  frontend:
  backend:
  
volumes:
  redis-data:
 ```

Zaženemo in pokažemo, da se podatki ohranijo:

    docker-compose -f docker-compose.prod.yml up
    docker-compose -f docker-compose.prod.yml down
    docker-compose -f docker-compose.prod.yml up
    docker-compose -f docker-compose.prod.yml down --volumes
    docker-compose -f docker-compose.prod.yml up